## Part 1: MNIST Softmax Classifier Demo in TensorFlow

TensorFlow is already installed on the SCC. Please review the instructions on [connecting to SCC](https://docs.google.com/document/d/1C4XrrTZIRfmJ6-LHmuJ4levTnunezujVrxCz8jJ4rec), and [running jobs on SCC](http://www.bu.edu/tech/support/research/system-usage/running-jobs/).

Make sure you are capable of running this demo (using `qsub`) on the SCC cluster: [`mnist_softmax_scope.py`](https://github.com/kunhe/cs591s2/blob/master/mnist_softmax_scope.py). There is no required deliverable, but this exercise is helpful for running jobs on the SCC in the future.

The demo is another implementation of `mnist_softmax.py` presented in the MNIST for ML Beginners tutorial, but using scopes. For the purposes of this assignment, the contents of the demo is also replicated below.

In [1]:
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def logistic_regression(x_):
    # create the actual model
    scope_args = {'initializer': tf.random_normal_initializer(stddev=1e-4)}
    with tf.variable_scope("weights", **scope_args):
        W = tf.get_variable('W', shape=[784, 10])
        b = tf.get_variable('b', shape=[10])
        y_logits = tf.matmul(x_, W) + b
    return y_logits

def test_classification(model_function, learning_rate=0.1):
    # import data
    mnist = input_data.read_data_sets('./datasets/mnist/', one_hot=True)

    with tf.Graph().as_default() as g:
        # where are you going to allocate memory and perform computations
        with tf.device("/cpu:0"):
            
            # define model "input placeholders", i.e. variables that are
            # going to be substituted with input data on train/test time
            x_ = tf.placeholder(tf.float32, [None, 784])
            y_ = tf.placeholder(tf.float32, [None, 10])
            y_logits = model_function(x_)
            
            # naive implementation of loss:
            # > losses = y_ * tf.log(tf.nn.softmax(y_logits))
            # > tf.reduce_mean(-tf.reduce_sum(losses, 1))
            # can be numerically unstable.
            #
            # so here we use tf.nn.softmax_cross_entropy_with_logits on the raw
            # outputs of 'y', and then average across the batch.
            
            losses = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_logits)
            cross_entropy_loss = tf.reduce_mean(losses)
            train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)
            
            y_pred = tf.argmax(tf.nn.softmax(y_logits), dimension=1)
            correct_prediction = tf.equal(y_pred, tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with g.as_default(), tf.Session() as sess:
        # that is how we "execute" statements 
        # (return None, e.g. init() or train_op())
        # or compute parts of graph defined above (loss, output, etc.)
        # given certain input (x_, y_)
        sess.run(tf.initialize_all_variables())
        
        # train
        for iter_i in range(20001):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            sess.run(train_step, feed_dict={x_: batch_xs, y_: batch_ys})
            
            # test trained model
            if iter_i % 2000 == 0:
                tf_feed_dict = {x_: mnist.test.images, y_: mnist.test.labels}
                acc_value = sess.run(accuracy, feed_dict=tf_feed_dict)
                print ('iteration %d\t accuracy: %.3f'%(iter_i, acc_value))
                
test_classification(logistic_regression, learning_rate=0.1)


Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
iteration 0	 accuracy: 0.405
iteration 2000	 accuracy: 0.918
iteration 4000	 accuracy: 0.920
iteration 6000	 accuracy: 0.922
iteration 8000	 accuracy: 0.921
iteration 10000	 accuracy: 0.925
iteration 12000	 accuracy: 0.923
iteration 14000	 accuracy: 0.923
iteration 16000	 accuracy: 0.923
iteration 18000	 accuracy: 0.924
iteration 20000	 accuracy: 0.925


## Part 2: Building Neural Networks in TensorFlow

(45 points)

### Q2.1 MLP in TensorFlow

Task: 

- Implement a multi-layer perceptron function **`mlp(x, hidden_sizes, activation_fn)`** in TensorFlow that has the following input and output: 

 **Inputs**
 - `x`, an input tensor of the images in the current batch `[batch_size, 28x28]`
 - `hidden_sizes`, a list of the number of hidden units per layer. For example: `[5,2]` means  5 hidden units in the first layer, and 2 hidden units in the second (output) layer.
 (Note: for MNIST, we need `hidden_sizes[-1]==10` since it has 10 classes.)
 - `activation_fn`, the activation function to be applied

 **Output**
 - a tensor of shape `[batch_size, hidden_sizes[-1]]`. 

**Note: ** 
- Make sure the activation function is not applied to the final (output) layer.
- It is recommended to use scopes and `tf.get_variable()` (as opposed to `tf.Variable()` which you may see elsewhere), as demonstrated in the sample code, and explained in the "Sharing Variables" tutorial in Part 0. Also see [here](http://stackoverflow.com/questions/37098546/difference-between-variable-and-get-variable-in-tensorflow).

In [4]:

def mlp(x, hidden_sizes, activation_fn):
    if not isinstance(hidden_sizes, (list, tuple)):
        raise ValueError("hidden_sizes must be a list or a tuple")
    scope_args = {'initializer': tf.random_normal_initializer(stddev=2e-1)}
    with tf.variable_scope("weights", **scope_args):
        hidden_sizes = [784] + hidden_sizes   
        h = x
        for i in range(len(hidden_sizes)-1):           
            with tf.variable_scope("hidden" + str(i)) as scope:
                temp = h
                W = tf.get_variable('W', shape=[hidden_sizes[i], hidden_sizes[i+1]])
                b = tf.get_variable('b', shape=hidden_sizes[i+1])
                h = activation_fn(tf.matmul(h, W) + b) 
                
        y_logits = tf.matmul(temp, W) + b
                     
    return y_logits
    

The following code tests your  implementation of the **`mlp()`** function. It basically recreates the 2-layer MLP with ReLU activation that you implemented in Problem Set 2. It should give an accuracy of above 0.97.

In [3]:
test_classification(lambda x: mlp(x, [64, 10], activation_fn=tf.nn.relu), learning_rate=0.1)

Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
iteration 0	 accuracy: 0.123
iteration 2000	 accuracy: 0.939
iteration 4000	 accuracy: 0.955
iteration 6000	 accuracy: 0.961
iteration 8000	 accuracy: 0.964
iteration 10000	 accuracy: 0.966
iteration 12000	 accuracy: 0.969
iteration 14000	 accuracy: 0.968
iteration 16000	 accuracy: 0.970
iteration 18000	 accuracy: 0.973
iteration 20000	 accuracy: 0.971


### Q2.2 Siamese Network
A siamese network is a network having two identical subnetworks that share parameters. A siamese network gives a *similarity metric* between pairs of inputs, which assigns high values to similar pairs and low values to dissimilar pairs. In MNIST, we can construct similar pairs by taking images from the same digit class, and dissimilar pairs from images from different classes. 


A sample Siamese network is presented below [Koch et al., Siamese Neural Networks for One-shot Image Recognition, ICML 2015]. Note how the two subnetworks have identical parameters:

<img src="siamese.png" style="width:480px;">


Given a Siamese network, we can determine the similarity between inputs $x_1$ and $x_2$, by taking the sign of the cosine similarity $\frac{\langle r_1,r_2\rangle}{\|r_1\|\|r_2\|}$, where $r_1$ and $r_2$ are the hidden representations produced by the network for $x_1$ and $x_2$, respectively. 

Tasks:

1. Build a Siamese Network in TensorFlow that receives as input two MNIST images, and decides whether they belong to the same digit category. Each subnetwork is an MLP with 2 hidden layers with 64 units, followed by a "distance layer" with 32 units. They will be created using the **`mlp()`** function you implemented earlier, using input argument `hidden_sizes=[64,64,32]`. The network computes the cosine similarity, as defined above, in the final output layer.

2. Train and test this model, and  report your test accuracy for this task.

You will need to implement the following two functions.

In [28]:
#################################################################
# Inputs: arr1 and arr2 have shape [batch_size, hidden_sizes[-1]]
# Output: return tensor of shape [batch_size, ], the cosine 
#         similarity betwwen arr1 and arr2
# 
# Hint: use tf.l2_normalize, tf.mul, tf.reduce_sum
################################################################
def cosine_similarity(arr1, arr2):
    ###################################
    arr1 = tf.nn.l2_normalize(arr1,dim=1)
    arr2 = tf.nn.l2_normalize(arr2,dim=1)
    return tf.reduce_sum(tf.multiply(arr1,arr2), axis = 1)
    ###################################

    
#################################################################
# Inputs: mlp_args is a dictionary of arguments to the mlp() 
#         function. 
#         Example: mlp_args = {'hidden_sizes':[64, 64, 32]}
#################################################################
def build_model(mlp_args):
    with tf.Graph().as_default() as g:
        with tf.device("/cpu:0"):
            x1 = tf.placeholder(tf.float32, [None, 784])
            x2 = tf.placeholder(tf.float32, [None, 784])
            y = tf.placeholder(tf.float32, [None])

            with tf.variable_scope("siamese") as var_scope:
                x_repr1 = mlp(x1, **mlp_args)  # hidden representation of x1
                var_scope.reuse_variables()    # weight sharing! 
                x_repr2 = mlp(x2, **mlp_args)  # hidden representation of x2
                logits = cosine_similarity(x_repr1, x_repr2) # similarity
            ###################################
            loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits))
            y_prob = tf.nn.sigmoid(logits)
            y_pred  = tf.nn.relu(tf.sign(logits))
            correct_prediction = tf.equal(y_pred , y)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

            ###################################
            
            # define scalar: loss 
            # define vector: y_prob as sigmoid(cosine_similarity)
            # define vector: y_pred as sign(cosine_similarity)
            # define scalar: accuracy as the fraction of correct predictions
            
    return {'graph': g, 'inputs': [x1, x2, y], 'pred': y_pred, 'logits': logits,
            'prob': y_prob, 'loss': loss, 'accuracy': accuracy, 'xrep' :x_repr1}
######################

The following code segment prepares minibatch training data.

In [6]:
# data preparation
def mnist_siamese_dataset_iterator(batch_size, dataset_name):
    assert dataset_name in ['train', 'test']
    assert batch_size > 0 or batch_size == -1 # -1 for entire dataset
    mnist = input_data.read_data_sets('./datasets/mnist/', one_hot=True)
    dataset = getattr(mnist, dataset_name)
    
    while True:
        if batch_size > 0:
            X1, y1 = dataset.next_batch(batch_size)
            X2, y2 = dataset.next_batch(batch_size)
            y = np.argmax(y1, axis=1) == np.argmax(y2, axis=1)
            yield X1, X2, y
        else:
            X1 = dataset.images
            idx = np.arange(len(X1))
            np.random.shuffle(idx)
            X2 = X1[idx]
            y1 = dataset.labels
            y2 = y1[idx]
            y = np.argmax(y1, axis=1) == np.argmax(y2, axis=1)
            yield X1, X2, y

Now let's run the training.

Note: if you have a good GPU, you could change `tf.device("/cpu:0")` to `tf.device("/gpu:0")` in your **`build_model()`** function to speed up training.

In [41]:
try:
    from itertools import izip as zip
except ImportError:
    print('This is Python 3')

def run_training(model_dict, train_data_iterator, test_full_iter, 
                 train_full_iter, n_iter, print_every=300):
    with model_dict['graph'].as_default():
        #optimizer = tf.train.AdagradOptimizer(learning_rate = 0.2)
        optimizer = tf.train.AdamOptimizer()
        #optimizer = tf.train.AdadeltaOptimizer(learning_rate = 0.2)
        train_op = optimizer.minimize(model_dict['loss'])
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for iter_i, data_batch in zip(range(n_iter), train_data_iterator):
                batch_feed_dict = dict(zip(model_dict['inputs'], data_batch))
                sess.run(train_op, feed_dict=batch_feed_dict)
                if iter_i % print_every == 0:
                    print_zip_iter = zip([test_full_iter, train_full_iter], ['train', 'test'])
                    for data_iterator, data_name in print_zip_iter:
                        test_batch = next(data_iterator)
                        batch_feed_dict = dict(zip(model_dict['inputs'], test_batch))
                        to_compute = [model_dict['accuracy'], model_dict['loss']]
                        acc_value, loss_val = sess.run(to_compute, batch_feed_dict)
                        fmt = (iter_i, acc_value, loss_val)
                        print(data_name, 'iteration %d\t accuracy: %.3f, loss: %.3f'%fmt)

train_data_iterator = mnist_siamese_dataset_iterator(100, 'train')
test_full_iter = mnist_siamese_dataset_iterator(-1, 'test')
train_full_iter = mnist_siamese_dataset_iterator(-1, 'train')

mlp_args = {'hidden_sizes':[64, 64, 32] , 'activation_fn' :tf.nn.relu}
model = build_model(mlp_args)
run_training(model, train_data_iterator, test_full_iter, train_full_iter,  n_iter = 10000)

This is Python 3
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
train iteration 0	 accuracy: 0.098, loss: 0.978
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
test iteration 0	 accuracy: 0.099, loss: 0.976
train iteration 300	 accuracy: 0.637, loss: 0.658
test iteration 300	 accuracy: 0.634, loss: 0.661
train iteration 600	 accuracy: 0.684, loss: 0.648
test iteration 600	 accuracy: 0.688, loss: 0.647
train iteration 900	 accuracy: 0.707

## Part 3: Model Variants

(30 points)

Tasks:
 
 1. Create an improved version of the model in part 2. You are welcome to use techniques covered in class, including: increasing network depth, varying activation functions, learning rates, optimizers, cost functions, regularization etc. 
 
 2. Report the test accuracy.

Note: you should **not** use convolutional layers for this assignment.

In [42]:
try:
    from itertools import izip as zip
except ImportError:
    print('This is Python 3')

def run_training(model_dict, train_data_iterator, test_full_iter, 
                 train_full_iter, n_iter, print_every=500):
    with model_dict['graph'].as_default():
        #optimizer = tf.train.AdagradOptimizer(learning_rate = 0.2)
        optimizer = tf.train.AdamOptimizer()
        #optimizer = tf.train.AdadeltaOptimizer(learning_rate = 0.2)
        train_op = optimizer.minimize(model_dict['loss'])
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for iter_i, data_batch in zip(range(n_iter), train_data_iterator):
                batch_feed_dict = dict(zip(model_dict['inputs'], data_batch))
                sess.run(train_op, feed_dict=batch_feed_dict)
                if iter_i % print_every == 0:
                    print_zip_iter = zip([test_full_iter, train_full_iter], ['train', 'test'])
                    for data_iterator, data_name in print_zip_iter:
                        test_batch = next(data_iterator)
                        batch_feed_dict = dict(zip(model_dict['inputs'], test_batch))
                        to_compute = [model_dict['accuracy'], model_dict['loss']]
                        acc_value, loss_val = sess.run(to_compute, batch_feed_dict)
                        fmt = (iter_i, acc_value, loss_val)
                        print(data_name, 'iteration %d\t accuracy: %.3f, loss: %.3f'%fmt)

train_data_iterator = mnist_siamese_dataset_iterator(100, 'train')
test_full_iter = mnist_siamese_dataset_iterator(-1, 'test')
train_full_iter = mnist_siamese_dataset_iterator(-1, 'train')

mlp_args = {'hidden_sizes':[64, 64,32, 32] , 'activation_fn' :tf.nn.relu}
model = build_model(mlp_args)
run_training(model, train_data_iterator, test_full_iter, train_full_iter,  n_iter = 15000)

This is Python 3
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
train iteration 0	 accuracy: 0.103, loss: 1.024
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
test iteration 0	 accuracy: 0.102, loss: 1.027
train iteration 500	 accuracy: 0.735, loss: 0.649
test iteration 500	 accuracy: 0.738, loss: 0.647
train iteration 1000	 accuracy: 0.794, loss: 0.638
test iteration 1000	 accuracy: 0.788, loss: 0.641
train iteration 1500	 accuracy: 0.

**Deeper Network:**
As we have seen above, when we make our network deeper our Acc is gonna be better and it is actually obvious because wwhen we have more hidden layer our performance is going to be better. 

In [44]:
try:
    from itertools import izip as zip
except ImportError:
    print('This is Python 3')

def run_training(model_dict, train_data_iterator, test_full_iter, 
                 train_full_iter, n_iter, print_every=500):
    with model_dict['graph'].as_default():
        #optimizer = tf.train.AdagradOptimizer(learning_rate = 0.2)
        optimizer = tf.train.AdamOptimizer()
        #optimizer = tf.train.AdadeltaOptimizer(learning_rate = 0.2)
        train_op = optimizer.minimize(model_dict['loss'])
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for iter_i, data_batch in zip(range(n_iter), train_data_iterator):
                batch_feed_dict = dict(zip(model_dict['inputs'], data_batch))
                sess.run(train_op, feed_dict=batch_feed_dict)
                if iter_i % print_every == 0:
                    print_zip_iter = zip([test_full_iter, train_full_iter], ['train', 'test'])
                    for data_iterator, data_name in print_zip_iter:
                        test_batch = next(data_iterator)
                        batch_feed_dict = dict(zip(model_dict['inputs'], test_batch))
                        to_compute = [model_dict['accuracy'], model_dict['loss']]
                        acc_value, loss_val = sess.run(to_compute, batch_feed_dict)
                        fmt = (iter_i, acc_value, loss_val)
                        print(data_name, 'iteration %d\t accuracy: %.3f, loss: %.3f'%fmt)

train_data_iterator = mnist_siamese_dataset_iterator(100, 'train')
test_full_iter = mnist_siamese_dataset_iterator(-1, 'test')
train_full_iter = mnist_siamese_dataset_iterator(-1, 'train')

mlp_args = {'hidden_sizes':[64, 64, 32] , 'activation_fn' :tf.nn.softplus}
model = build_model(mlp_args)
run_training(model, train_data_iterator, test_full_iter, train_full_iter,  n_iter = 10000)

This is Python 3
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
train iteration 0	 accuracy: 0.102, loss: 1.039
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
test iteration 0	 accuracy: 0.100, loss: 1.039
train iteration 500	 accuracy: 0.654, loss: 0.654
test iteration 500	 accuracy: 0.658, loss: 0.654
train iteration 1000	 accuracy: 0.715, loss: 0.643
test iteration 1000	 accuracy: 0.716, loss: 0.642
train iteration 1500	 accuracy: 0.

**Changing Activation Function:**
In this part I just replace relu with softplus and then I get almostly the same result with relu(as I have explained before in pset2). 

In [45]:
def run_training(model_dict, train_data_iterator, test_full_iter, 
                 train_full_iter, n_iter, print_every=500):
    with model_dict['graph'].as_default():
        optimizer = tf.train.AdagradOptimizer(learning_rate = 0.2)
        #optimizer = tf.train.AdamOptimizer()
        #optimizer = tf.train.AdadeltaOptimizer(learning_rate = 0.2)
        train_op = optimizer.minimize(model_dict['loss'])
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for iter_i, data_batch in zip(range(n_iter), train_data_iterator):
                batch_feed_dict = dict(zip(model_dict['inputs'], data_batch))
                sess.run(train_op, feed_dict=batch_feed_dict)
                if iter_i % print_every == 0:
                    print_zip_iter = zip([test_full_iter, train_full_iter], ['train', 'test'])
                    for data_iterator, data_name in print_zip_iter:
                        test_batch = next(data_iterator)
                        batch_feed_dict = dict(zip(model_dict['inputs'], test_batch))
                        to_compute = [model_dict['accuracy'], model_dict['loss']]
                        acc_value, loss_val = sess.run(to_compute, batch_feed_dict)
                        fmt = (iter_i, acc_value, loss_val)
                        print(data_name, 'iteration %d\t accuracy: %.3f, loss: %.3f'%fmt)

train_data_iterator = mnist_siamese_dataset_iterator(100, 'train')
test_full_iter = mnist_siamese_dataset_iterator(-1, 'test')
train_full_iter = mnist_siamese_dataset_iterator(-1, 'train')

mlp_args = {'hidden_sizes':[64, 64, 32] , 'activation_fn' :tf.nn.relu}
model = build_model(mlp_args)
run_training(model, train_data_iterator, test_full_iter, train_full_iter,  n_iter = 10000)

Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
train iteration 0	 accuracy: 0.119, loss: 0.869
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
test iteration 0	 accuracy: 0.121, loss: 0.865
train iteration 500	 accuracy: 0.668, loss: 0.656
test iteration 500	 accuracy: 0.665, loss: 0.655
train iteration 1000	 accuracy: 0.733, loss: 0.648
test iteration 1000	 accuracy: 0.738, loss: 0.642
train iteration 1500	 accuracy: 0.773, loss: 0.638


**Changing Optimizer: **
In this part we replace AdamOptimizer() with AdagradOptimizer() and we are seeing that it works a little bit worse and it converges a bit slower. 

In [47]:
def build_model(mlp_args):
    with tf.Graph().as_default() as g:
        with tf.device("/cpu:0"):
            x1 = tf.placeholder(tf.float32, [None, 784])
            x2 = tf.placeholder(tf.float32, [None, 784])
            y = tf.placeholder(tf.float32, [None])

            with tf.variable_scope("siamese") as var_scope:
                x_repr1 = mlp(x1, **mlp_args)  # hidden representation of x1
                var_scope.reuse_variables()    # weight sharing! 
                x_repr2 = mlp(x2, **mlp_args)  # hidden representation of x2
                logits = cosine_similarity(x_repr1, x_repr2) # similarity
            ###################################
            loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits))
            reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
            loss = loss + 0.01 * sum(reg_losses)
            y_prob = tf.nn.sigmoid(logits)
            y_pred  = tf.nn.relu(tf.sign(logits))
            correct_prediction = tf.equal(y_pred , y)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

            ###################################
            
            # define scalar: loss 
            # define vector: y_prob as sigmoid(cosine_similarity)
            # define vector: y_pred as sign(cosine_similarity)
            # define scalar: accuracy as the fraction of correct predictions
            
    return {'graph': g, 'inputs': [x1, x2, y], 'pred': y_pred, 'logits': logits,
            'prob': y_prob, 'loss': loss, 'accuracy': accuracy, 'xrep' :x_repr1}

try:
    from itertools import izip as zip
except ImportError:
    print('This is Python 3')

def run_training(model_dict, train_data_iterator, test_full_iter, 
                 train_full_iter, n_iter, print_every=500):
    with model_dict['graph'].as_default():
        #optimizer = tf.train.AdagradOptimizer(learning_rate = 0.2)
        optimizer = tf.train.AdamOptimizer()
        #optimizer = tf.train.AdadeltaOptimizer(learning_rate = 0.2)
        train_op = optimizer.minimize(model_dict['loss'])
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for iter_i, data_batch in zip(range(n_iter), train_data_iterator):
                batch_feed_dict = dict(zip(model_dict['inputs'], data_batch))
                sess.run(train_op, feed_dict=batch_feed_dict)
                if iter_i % print_every == 0:
                    print_zip_iter = zip([test_full_iter, train_full_iter], ['train', 'test'])
                    for data_iterator, data_name in print_zip_iter:
                        test_batch = next(data_iterator)
                        batch_feed_dict = dict(zip(model_dict['inputs'], test_batch))
                        to_compute = [model_dict['accuracy'], model_dict['loss']]
                        acc_value, loss_val = sess.run(to_compute, batch_feed_dict)
                        fmt = (iter_i, acc_value, loss_val)
                        print(data_name, 'iteration %d\t accuracy: %.3f, loss: %.3f'%fmt)

train_data_iterator = mnist_siamese_dataset_iterator(100, 'train')
test_full_iter = mnist_siamese_dataset_iterator(-1, 'test')
train_full_iter = mnist_siamese_dataset_iterator(-1, 'train')

mlp_args = {'hidden_sizes':[64, 64, 32] , 'activation_fn' :tf.nn.relu}
model = build_model(mlp_args)
run_training(model, train_data_iterator, test_full_iter, train_full_iter,  n_iter = 15000)

This is Python 3
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
train iteration 0	 accuracy: 0.103, loss: 1.018
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
test iteration 0	 accuracy: 0.099, loss: 1.021
train iteration 500	 accuracy: 0.684, loss: 0.648
test iteration 500	 accuracy: 0.680, loss: 0.649
train iteration 1000	 accuracy: 0.732, loss: 0.640
test iteration 1000	 accuracy: 0.732, loss: 0.638
train iteration 1500	 accuracy: 0.

**Regulirization:** 
Here we add regulirization to our loss function. 

## Part 4: Visualize learned features
(25 points)

Once a neural network is trained, we can think of the last layer as performing prediction, and the activations from layer before  become the input "features" to the predictor. If the neural network is performing good predictions, then this means that these activations encode useful features that are 1) representative of the input, and 2) discriminative for the prediction task. Activations at a selected layer could then be used as generic feature encodings of the input. 

We  expect a "good" feature encoding scheme to group similar inputs together in the feature encoding space. To check that, we can visualize the features in 2-dimensional space and check if similar examples (in this case, sharing the same labels) are close together. 

Tasks:

1. In TensorFlow, extract 32-dimensional features from the **distance layer** of your trained Siamese network, for the MNIST **test set**.
2. Reduce the dimensionality of your deep features to 2 using [t-SNE embedding](https://lvdmaaten.github.io/tsne/). You may use [this fast implementation](https://github.com/lvdmaaten/bhtsne/) with correct attribution. You could alternatively use [sklearn.manifold.TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), but please take note that it might be slow.
3. Visualize the 2-dimensional embeddings. If your extracted features are good, data points representing a specific digit should appear within a compact cluster. In the example below, each color corresponds to a digit class.

 <img src="tsne.png" style="width:480px;">
 
For this part, the starter code is in minimalist fashion. Good luck!

In [39]:
import bhtsne
import pylab as pl

def visualize(features):
    tsne2 = bhtsne.run_bh_tsne(features[1:10,:], initial_dims=features.shape[1])
    pl.plot(tsne2[0,:], tnse2[1,:])
    pl.show()

In [40]:
def run_training(model_dict, train_data_iterator, test_full_iter, 
                 train_full_iter, n_iter, print_every=300):
    with model_dict['graph'].as_default():
        optimizer = tf.train.AdagradOptimizer(learning_rate = 0.2)
        #optimizer = tf.train.AdamOptimizer()
        #optimizer = tf.train.AdadeltaOptimizer(learning_rate = 0.2)
        train_op = optimizer.minimize(model_dict['loss'])
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for iter_i, data_batch in zip(range(n_iter), train_data_iterator):
                batch_feed_dict = dict(zip(model_dict['inputs'], data_batch))
                sess.run(train_op, feed_dict=batch_feed_dict)
                if iter_i % print_every == 0:
                    print_zip_iter = zip([test_full_iter, train_full_iter], ['train', 'test'])
                    for data_iterator, data_name in print_zip_iter:
                        test_batch = next(data_iterator)
                        batch_feed_dict = dict(zip(model_dict['inputs'], test_batch))
                        to_compute = [model_dict['accuracy'], model_dict['loss']]
                        acc_value, loss_val = sess.run(to_compute, batch_feed_dict)
                        fmt = (iter_i, acc_value, loss_val)
                        print(data_name, 'iteration %d\t accuracy: %.3f, loss: %.3f'%fmt)
                        features = sess.run(model_dict['xrep'], batch_feed_dict)
            visualize(features)

train_data_iterator = mnist_siamese_dataset_iterator(100, 'train')
test_full_iter = mnist_siamese_dataset_iterator(-1, 'test')
train_full_iter = mnist_siamese_dataset_iterator(-1, 'train')

mlp_args = {'hidden_sizes':[64, 64, 32] , 'activation_fn' :tf.nn.relu}
model = build_model(mlp_args)
run_training(model, train_data_iterator, test_full_iter, train_full_iter,  n_iter = 1000)

Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
train iteration 0	 accuracy: 0.180, loss: 0.821
Extracting ./datasets/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist/t10k-labels-idx1-ubyte.gz
test iteration 0	 accuracy: 0.179, loss: 0.821
train iteration 300	 accuracy: 0.627, loss: 0.661
test iteration 300	 accuracy: 0.630, loss: 0.659
train iteration 600	 accuracy: 0.663, loss: 0.653
test iteration 600	 accuracy: 0.664, loss: 0.651
train iteration 900	 accuracy: 0.664, loss: 0.650
tes

/home/siaa/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Please run this program directly from python and not from ipython or jupyter.
This is an issue due to asynchronous error handling.


ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/home/siaa/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/home/siaa/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/siaa/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 276, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home/siaa/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 228, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/siaa/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 412, in execute_request
    self._abort_queues()
  File "/home/siaa/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 628, in _abort_queues
    self._abort_queue(stream)
  File "/home/siaa/anaconda3/lib/python3.6/site

Traceback (most recent call last):
  File "/home/siaa/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-348e2dd0cfac>", line 31, in <module>
    run_training(model, train_data_iterator, test_full_iter, train_full_iter,  n_iter = 1000)
  File "<ipython-input-40-348e2dd0cfac>", line 23, in run_training
    visualize(features)
  File "<ipython-input-39-282f7cebdea1>", line 5, in visualize
    tsne2 = bhtsne.run_bh_tsne(features[1:10,:], initial_dims=features.shape[1])
  File "/home/siaa/Downloads/MehrnooshDeepLearn/cs591s2-master/bhtsne.py", line 214, in run_bh_tsne
    for result in bh_tsne(tmp_dir_path, verbose):
  File "/home/siaa/Downloads/MehrnooshDeepLearn/cs591s2-master/bhtsne.py", line 156, in bh_tsne
    'refer to the bh_tsne output for further details')
AssertionError: ERROR: Call to bh_tsne exited with a non-zero return code exit status, please enable ve

AssertionError: ERROR: Call to bh_tsne exited with a non-zero return code exit status, please enable verbose mode and refer to the bh_tsne output for further details